# **CASE: *TURNOVER* DE FUNCIONÁRIOS**

---

### Instalação de bibliotecas

In [ ]:
# Para cálculo do IV
install.packages("Information")
# Para categorização de variáveis
install.packages("gtools")
# Para cálculo do VIF
install.packages("rms")
# Para cálculo do KS e AUC
install.packages("ROCR")

### Carregamento de bibliotecas

In [ ]:
library(tidyverse) # já pré-instalado, se for em Google Colab
library(Information)
library(gtools)
library(rms)
library(ROCR)

### Leitura das bases de dados

In [ ]:
dados_turnover_jan <- read.table(file = "Turnover_Funcionarios_Jan_23.txt",
                                 sep = "\t",
                                 dec = ".",
                                 header = TRUE)

In [ ]:
dados_turnover_jul <- read.table(file = "Turnover_Funcionarios_Jul_23.txt",
                                 sep = "\t",
                                 dec = ".",
                                 header = TRUE)

### Tamanho das bases de dados

In [ ]:
dim(dados_turnover_jan)

In [ ]:
dim(dados_turnover_jul)

### Análise exploratória breve (safra de jan/23)

In [ ]:
# Medidas resumo
summary(dados_turnover_jan)

In [ ]:
# Tabela de frequências: departamento
table(dados_turnover_jan$departamento, useNA = "always")
prop.table(table(dados_turnover_jan$departamento, useNA = "always"))

In [ ]:
# Tabela de frequências: patamar de salário
table(dados_turnover_jan$patamar_salario, useNA = "always")
prop.table(table(dados_turnover_jan$patamar_salario, useNA = "always"))

### Análise do poder preditivo das variáveis (IV)

In [ ]:
# Comando auxiliar para para omitir notação científica nos p-valores e controlar largura dos outputs na tela do Colab
options(scipen = 999, width = 200)

In [ ]:
# Cálculo e exibição dos IV
IV <- create_infotables(data = dados_turnover_jan[,c(2:17)],
                        y = "turnover")
IV$Summary

### Categorização de variáveis com valores *missing* (safra de jan/23)

In [ ]:
# Cenários de categorização da variável 'ult_avaliacao_clima', com diferentes quantidades de grupos
prop.table(table(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 3), dados_turnover_jan$turnover, useNA = "always"), 1)
prop.table(table(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 4), dados_turnover_jan$turnover, useNA = "always"), 1)
prop.table(table(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 5), dados_turnover_jan$turnover, useNA = "always"), 1)
prop.table(table(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 6), dados_turnover_jan$turnover, useNA = "always"), 1)

In [ ]:
# Criação de variável 'ult_avaliacao_clima_cat' com cenário escolhido de categorização
dados_turnover_jan$ult_avaliacao_clima_cat <- as.factor(quantcut(dados_turnover_jan$ult_avaliacao_clima, q = 3))

In [ ]:
# Criação de categoria 'Vazio' na nova variável 'ult_avaliacao_clima_cat'
levels(dados_turnover_jan$ult_avaliacao_clima_cat) <- c(levels(dados_turnover_jan$ult_avaliacao_clima_cat), "Vazio")
dados_turnover_jan$ult_avaliacao_clima_cat[is.na(dados_turnover_jan$ult_avaliacao_clima_cat)] <- "Vazio"

### Separação de conjuntos de treino e teste

In [ ]:
# Aletorizando a ordem das linhas da base
set.seed(12345)
dados_turnover_jan = sample_n(dados_turnover_jan, size = nrow(dados_turnover_jan))

In [ ]:
# Definindo tamanho do conjunto de treino
tamanho_treino <- floor(nrow(dados_turnover_jan) * 0.7)

In [ ]:
# Separando os conjuntos de treino e teste
dados_turnover_treino <- dados_turnover_jan[1:tamanho_treino,]
dados_turnover_teste  <- dados_turnover_jan[(tamanho_treino+1):nrow(dados_turnover_jan),]

### Modelo de regressão logística múltipla

In [ ]:
# Ajuste do modelo 1: inicial
regressao_1 <- glm(turnover ~
                     ult_avaliacao_clima_cat +
                     qtde_projetos_24m +
                     media_horas_trabalho_6m +
                     tempo_empresa +
                     flag_promocao_3m +
                     departamento +
                     patamar_salario,
                   family = binomial (link = 'logit'),
                   data = dados_turnover_treino)

summary(regressao_1)

In [ ]:
# Ajuste do modelo 2: retirando 'media_horas_trabalho_6m'
regressao_2 <- glm(turnover ~
                     ult_avaliacao_clima_cat +
                     qtde_projetos_24m +
                     tempo_empresa +
                     flag_promocao_3m +
                     departamento +
                     patamar_salario,
                   family = binomial (link = 'logit'),
                   data = dados_turnover_treino)

summary(regressao_2)

In [ ]:
# Ajuste do modelo 3: retirando 'qtde_projetos_24m'
regressao_3 <- glm(turnover ~
                     ult_avaliacao_clima_cat +
                     tempo_empresa +
                     flag_promocao_3m +
                     departamento +
                     patamar_salario,
                   family = binomial (link = 'logit'),
                   data = dados_turnover_treino)

summary(regressao_3)

In [ ]:
# Avaliação de colinearidade no modelo 3 (é natural que os VIF sejam altos para dummies de uma mesma variável qualitativa)
data.frame(VIF = vif(regressao_3))

### Aplicação do modelo no conjunto de treino

In [ ]:
# Aplicação do modelo no conjunto de treino (criação de uma nova coluna chamada "probabilidade")
dados_turnover_treino$probabilidade <- predict(regressao_3,
                                               dados_turnover_treino,
                                               type = "response")

In [ ]:
# Definição de ponto de corte (padrão: % de 1's na amostra)
ponto_corte <- mean(dados_turnover_treino$turnover)
ponto_corte

In [ ]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_turnover_treino$predito <- as.factor(ifelse(dados_turnover_treino$probabilidade > ponto_corte, 1, 0))

### Análise de desempenho no conjunto de treino

In [ ]:
# Tabela de classificação
tabela <- table(dados_turnover_treino$turnover, dados_turnover_treino$predito)

In [ ]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

In [ ]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

In [ ]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

In [ ]:
# KS
pred <- prediction(dados_turnover_treino$probabilidade, dados_turnover_treino$turnover)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

In [ ]:
# AUC
pred <- prediction(dados_turnover_treino$probabilidade, dados_turnover_treino$turnover)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

### Aplicação do modelo no conjunto de teste

In [ ]:
# Aplicação do modelo no conjunto de teste (criação de uma nova coluna chamada "probabilidade")
dados_turnover_teste$probabilidade <- predict(regressao_3,
                                              dados_turnover_teste,
                                              type = "response")

In [ ]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_turnover_teste$predito <- as.factor(ifelse(dados_turnover_teste$probabilidade > ponto_corte, 1, 0))

### Análise de desempenho no conjunto de teste

In [ ]:
# Tabela de classificação
tabela <- table(dados_turnover_teste$turnover, dados_turnover_teste$predito)

In [ ]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

In [ ]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

In [ ]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

In [ ]:
# KS
pred <- prediction(dados_turnover_teste$probabilidade, dados_turnover_teste$turnover)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

In [ ]:
# AUC
pred <- prediction(dados_turnover_teste$probabilidade, dados_turnover_teste$turnover)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)

### Aplicação do modelo na base de validação (safra de jul/23)

In [ ]:
# Criação de variável 'ult_avaliacao_clima_cat' com cenário escolhido de categorização
dados_turnover_jul$ult_avaliacao_clima_cat <- as.factor(case_when(dados_turnover_jul$ult_avaliacao_clima <= 0.51 ~ "[0,0.51]",
                                                                  dados_turnover_jul$ult_avaliacao_clima <= 0.83 ~ "(0.51,0.83]",
                                                                  dados_turnover_jul$ult_avaliacao_clima <= 1    ~ "(0.83,1]",
                                                                  is.na(dados_turnover_jul$ult_avaliacao_clima)  ~ "Vazio"))

In [ ]:
# Aplicação do modelo no conjunto de teste (criação de uma nova coluna chamada "probabilidade")
dados_turnover_jul$probabilidade <- predict(regressao_3,
                                            dados_turnover_jul,
                                            type = "response")

In [ ]:
# Definição da resposta predita pelo modelo (criação de uma nova coluna chamada "predito")
dados_turnover_jul$predito <- as.factor(ifelse(dados_turnover_jul$probabilidade > ponto_corte, 1, 0))

### Análise de desempenho na base de validação

In [ ]:
# Tabela de classificação
tabela <- table(dados_turnover_jul$turnover, dados_turnover_jul$predito)

In [ ]:
# Acurácia
(tabela[1,1] + tabela[2,2]) / sum(tabela)

In [ ]:
# Especificidade
tabela[1,1] / (tabela[1,1] + tabela[1,2])

In [ ]:
# Sensibilidade
tabela[2,2] / (tabela[2,1] + tabela[2,2])

In [ ]:
# KS
pred <- prediction(dados_turnover_jul$probabilidade, dados_turnover_jul$turnover)
perf <- performance(pred, "tpr", "fpr")
ks <- max(attr(perf, 'y.values')[[1]] - attr(perf, 'x.values')[[1]])

print(ks)

In [ ]:
# AUC
pred <- prediction(dados_turnover_jul$probabilidade, dados_turnover_jul$turnover)
auc <- performance(pred, "auc")
auc <- auc@y.values[[1]]

print(auc)